In [ ]:
# #1 Kütüphaneleri içe aktarma
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import hamming_loss, classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
import joblib
import os
from time import time
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Görselleştirme için klasör oluştur
results_dir = "C:\\Users\\DangerClose\\Desktop\\Stack Overflow Q&A\\RandomForest_results"
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [ ]:
# #2 Görsel ayarları yapma
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(font_scale=1.1)
colors = sns.color_palette("mako", 8)

In [ ]:
# #3 Veri yükleme
print("1. Veri yükleniyor...")
data = pd.read_csv('C:\\Users\\DangerClose\\Desktop\\Stack Overflow Q&A\\model_ready_sample.csv')
print(f"Toplam örnek sayısı: {len(data)}")


In [ ]:
# #4 Etiket sütunlarını belirleme
tag_columns = [col for col in data.columns if col.startswith('tag_')]
print(f"Toplam etiket sayısı: {len(tag_columns)}")
print(f"Etiketler: {', '.join(tag_columns)}")

# Etiket dağılımını analiz edelim
tag_counts = {}
for col in tag_columns:
    tag_name = col.replace('tag_', '')
    tag_counts[tag_name] = data[col].sum()

tag_counts_df = pd.DataFrame({'tag': list(tag_counts.keys()), 
                             'count': list(tag_counts.values())})
tag_counts_df = tag_counts_df.sort_values('count', ascending=False)

In [ ]:
# #5 En sık kullanılan etiketleri görselleştirme
plt.figure(figsize=(12, 8))
sns.barplot(x='count', y='tag', data=tag_counts_df, palette='viridis')
plt.title('Etiket Dağılımı', fontsize=16)
plt.xlabel('Soru Sayısı', fontsize=14)
plt.ylabel('Etiket', fontsize=14)
plt.tight_layout()
plt.savefig('tag_distribution.png')
plt.show()

In [ ]:
# #6 Özellik ve hedef değişkenleri hazırlama
print("2. Veri hazırlanıyor...")
# Metin sütununu seçme (işlenmiş gövde metni kullanılacak)
X = data['processed_body'] if 'processed_body' in data.columns else data['complete_text']
print(f"Kullanılan metin özelliği: {X.name}")

# Etiket matrisini oluştur
y = data[tag_columns].values


In [ ]:
# #7 Veriyi eğitim ve test setlerine ayırma
print("3. Veri bölünüyor (eğitim/test)...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=np.sum(y, axis=1) > 1)

print(f"Eğitim seti: {X_train.shape[0]} örnek")
print(f"Test seti: {X_test.shape[0]} örnek")

In [ ]:
# #8 Metin vektörleştirme
print("4. Metin vektörleştirme yapılıyor...")
tfidf_vectorizer = TfidfVectorizer(
    max_features=10000, 
    min_df=5,
    max_df=0.8,
    ngram_range=(1, 2),
    sublinear_tf=True
)

In [ ]:
# #9 TF-IDF dönüşümü
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print(f"Özellik sayısı: {X_train_tfidf.shape[1]}")

In [ ]:
# #10 Özellik dağılımı görselleştirme
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_sum = np.array(X_train_tfidf.sum(axis=0)).flatten()
feature_importance = pd.DataFrame({'feature': feature_names, 'importance': tfidf_sum})
feature_importance = feature_importance.sort_values('importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_importance.head(20), palette='rocket')
plt.title('En Önemli 20 TF-IDF Özelliği', fontsize=16)
plt.xlabel('Toplam TF-IDF Skoru', fontsize=14)
plt.ylabel('Özellik', fontsize=14)
plt.tight_layout()
plt.savefig('top_tfidf_features.png')
plt.show()

In [ ]:
# #11 RandomForest sınıflandırıcısını konfigüre etme
print("5. RandomForest sınıflandırıcı konfigüre ediliyor...")
base_classifier = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    bootstrap=True,
    class_weight='balanced_subsample',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

In [ ]:
# #12 Hiperparametre etki analizi
tree_nums = [10, 50, 100, 200]
depths = [5, 10, 15, None]
results = []

for n_trees in tree_nums:
    for depth in depths:
        print(f"\nDenemeler: {n_trees} ağaç, max_depth={depth}")
        clf = RandomForestClassifier(
            n_estimators=n_trees,
            max_depth=depth,
            min_samples_split=10,
            min_samples_leaf=5,
            max_features='sqrt',
            bootstrap=True,
            class_weight='balanced_subsample',
            n_jobs=-1,
            random_state=42
        )
        
        # #13 Çok-etiketli model kurma
        model = MultiOutputClassifier(clf, n_jobs=-1)
        
        # #14 Model eğitimi
        t0 = time()
        model.fit(X_train_tfidf, y_train)
        train_time = time() - t0
        
        # #15 Test
        t0 = time()
        y_pred = model.predict(X_test_tfidf)
        test_time = time() - t0
        
        # #16 Performans metrikleri hesaplama
        h_loss = hamming_loss(y_test, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_test.ravel(), y_pred.ravel(), average='weighted', zero_division=0
        )
        
        results.append({
            'n_trees': n_trees,
            'max_depth': 'Unlimited' if depth is None else depth,
            'hamming_loss': h_loss,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'train_time': train_time,
            'test_time': test_time
        })
        
        print(f"Hamming Loss: {h_loss:.4f}, F1: {f1:.4f}, Eğitim süresi: {train_time:.2f}s")

In [ ]:
# #17 Hiperparametre sonuçlarını görselleştirme
results_df = pd.DataFrame(results)

plt.figure(figsize=(16, 12))

In [ ]:
# #17 Hiperparametre sonuçlarını görselleştirme
results_df = pd.DataFrame(results)

plt.figure(figsize=(16, 12))

# F1 Score ısı haritası
plt.subplot(2, 2, 1)
pivoted = pd.pivot_table(results_df, values="f1", index="n_trees", columns="max_depth")
sns.heatmap(pivoted, annot=True, fmt=".3f", cmap="YlGnBu")
plt.title('F1 Score', fontsize=16)

# Hamming Loss ısı haritası
plt.subplot(2, 2, 2)
pivoted = pd.pivot_table(results_df, values="hamming_loss", index="n_trees", columns="max_depth")
sns.heatmap(pivoted, annot=True, fmt=".3f", cmap="YlGnBu_r")
plt.title('Hamming Loss (düşük = iyi)', fontsize=16)

# Eğitim süresi ısı haritası
plt.subplot(2, 2, 3)
pivoted = pd.pivot_table(results_df, values="train_time", index="n_trees", columns="max_depth")
sns.heatmap(pivoted, annot=True, fmt=".1f", cmap="YlOrRd")
plt.title('Eğitim Süresi (saniye)', fontsize=16)

# Doğruluk/Duyarlılık ilişkisi grafiği
plt.subplot(2, 2, 4)
sns.scatterplot(
    x='precision', 
    y='recall', 
    size='n_trees',
    hue='max_depth',
    data=results_df,
    sizes=(100, 500),
    alpha=0.7
)
plt.title('Kesinlik vs Duyarlılık', fontsize=16)
plt.grid(True)

plt.tight_layout()
plt.savefig('hyperparameter_effects.png')
plt.show()



In [ ]:
# #18 En iyi sonuçlara göre model seçimi
best_result = results_df.loc[results_df['f1'].idxmax()]
print(f"\nEn iyi performans: {best_result['n_trees']} ağaç, max_depth={best_result['max_depth']}")
print(f"F1 Score: {best_result['f1']:.4f}, Hamming Loss: {best_result['hamming_loss']:.4f}")


In [ ]:
# #19 En iyi modeli oluşturma
print("\n6. En iyi RandomForest modeli eğitiliyor...")
best_max_depth = None if best_result['max_depth'] == 'Unlimited' else int(best_result['max_depth'])
best_clf = RandomForestClassifier(
    n_estimators=int(best_result['n_trees']),
    max_depth=best_max_depth,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    bootstrap=True,
    class_weight='balanced_subsample',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

In [ ]:
# #20 Çok-etiketli model oluşturma
print("\n7. Çok-etiketli model oluşturuluyor...")
final_model = MultiOutputClassifier(best_clf, n_jobs=-1)

In [ ]:
# #21 Final modeli eğitme
print("\n8. Model eğitiliyor...")
t0 = time()
final_model.fit(X_train_tfidf, y_train)
train_time = time() - t0
print(f"Model eğitimi tamamlandı. Süre: {train_time:.2f} saniye ({train_time/60:.2f} dakika)")

In [ ]:
# #22 Modeli kaydetme
print("\n9. Model kaydediliyor...")
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)

model_path = os.path.join(model_dir, 'randomforest_model.joblib')
vectorizer_path = os.path.join(model_dir, 'tfidf_vectorizer.joblib')

joblib.dump(final_model, model_path)
joblib.dump(tfidf_vectorizer, vectorizer_path)
print(f"Model kaydedildi: {model_path}")
print(f"Vektörleştirici kaydedildi: {vectorizer_path}")

In [ ]:
# #23 Final model testi
print("\n10. Model test ediliyor...")
t0 = time()
y_pred = final_model.predict(X_test_tfidf)
predict_time = time() - t0
print(f"Tahmin süresi: {predict_time:.2f} saniye (örnek başına {predict_time*1000/len(y_test):.2f} ms)")

In [ ]:
# #24 Performans değerlendirmesi
print("\n11. Performans değerlendirmesi...")
h_loss = hamming_loss(y_test, y_pred)
print(f"Hamming Loss: {h_loss:.4f} (düşük olması iyidir)")

In [ ]:
# #25 Etiket bazlı metrik hesaplama
tag_names = [col.replace('tag_', '') for col in tag_columns]
metrics = []

for i, tag_name in enumerate(tag_names):
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test[:, i], y_pred[:, i], average='binary', zero_division=0
    )
    
    accuracy = (y_test[:, i] == y_pred[:, i]).mean()
    support = y_test[:, i].sum()
    
    metrics.append({
        'Tag': tag_name,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'Accuracy': accuracy,
        'Support': support
    })

metrics_df = pd.DataFrame(metrics)

In [ ]:
# #26 En iyi ve en kötü performans gösteren etiketleri gösterme
print("\nEn yüksek F1 skoruna sahip 10 etiket:")
print(metrics_df.sort_values('F1-Score', ascending=False).head(10).to_string(index=False))

print("\nEn düşük F1 skoruna sahip 5 etiket:")
print(metrics_df.sort_values('F1-Score').head(5).to_string(index=False))

In [ ]:
# #27 Ortalama metrikleri hesaplama
print("\nOrtalama Metrikler:")
print(f"Precision: {metrics_df['Precision'].mean():.4f}")
print(f"Recall: {metrics_df['Recall'].mean():.4f}")
print(f"F1-Score: {metrics_df['F1-Score'].mean():.4f}")
print(f"Accuracy: {metrics_df['Accuracy'].mean():.4f}")


In [ ]:
# #28 Karışıklık matrisleri oluşturma
print("\n12. Örnek etiketler için karışıklık matrisleri oluşturuluyor...")
top_tag_indices = np.argsort([metrics_df[metrics_df['Tag'] == tag]['Support'].values[0] 
                              for tag in tag_names])[-5:]

plt.figure(figsize=(20, 15))
for i, idx in enumerate(top_tag_indices):
    tag = tag_names[idx]
    
    cm = confusion_matrix(y_test[:, idx], y_pred[:, idx])
    
    plt.subplot(2, 3, i+1)
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', 
                xticklabels=['Negatif', 'Pozitif'],
                yticklabels=['Negatif', 'Pozitif'])
    plt.title(f'Etiket: {tag}', fontsize=14)
    plt.ylabel('Gerçek Değer', fontsize=12)
    plt.xlabel('Tahmin', fontsize=12)
    plt.tight_layout()

plt.savefig(os.path.join(model_dir, 'confusion_matrices.png'))
plt.show()

In [ ]:
# #29 En iyi etiketler için F1 skoru görselleştirme
plt.figure(figsize=(12, 8))
top_metrics = metrics_df.sort_values('F1-Score', ascending=False).head(15)
sns.barplot(x='Tag', y='F1-Score', data=top_metrics, palette='viridis')
plt.title('En Yüksek F1 Skoruna Sahip 15 Etiket (RandomForest)', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig(os.path.join(model_dir, 'top_performing_tags.png'))
plt.show()

In [ ]:
# #30 Özellik önemi analizi
print("\n13. Özellik önemi analizi yapılıyor...")
top_tag = metrics_df.sort_values('Support', ascending=False).iloc[0]['Tag']
top_tag_idx = tag_names.index(top_tag)

if hasattr(final_model.estimators_[top_tag_idx], 'feature_importances_'):
    importances = final_model.estimators_[top_tag_idx].feature_importances_
    indices = np.argsort(importances)[-20:]
    
    feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
    top_features = [(feature_names[i], importances[i]) for i in indices[::-1]]
    
    print(f"\n'{top_tag}' etiketi için en önemli 10 özellik:")
    for feature, importance in top_features[:10]:
        print(f"{feature}: {importance:.4f}")
    
    plt.figure(figsize=(12, 8))
    plt.barh([feature_names[i] for i in indices[::-1]], 
             [importances[i] for i in indices[::-1]], 
             color=sns.color_palette('viridis', 20))
    plt.title(f"'{top_tag}' Etiketi İçin En Önemli 20 Özellik", fontsize=16)
    plt.xlabel('Önem Skoru', fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(model_dir, f'feature_importance_{top_tag}.png'))
    plt.show()

In [ ]:
# #31 Tahmin edilen ve gerçek etiketlerin dağılımı görselleştirme
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
predicted_counts = np.sum(y_pred, axis=0)
plt.bar(tag_names, predicted_counts, color=sns.color_palette('Set3', len(tag_names)))
plt.title('Tahmin Edilen Etiket Dağılımı', fontsize=14)
plt.xticks(rotation=90)
plt.ylabel('Soru Sayısı', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.subplot(1, 2, 2)
actual_counts = np.sum(y_test, axis=0)
plt.bar(tag_names, actual_counts, color=sns.color_palette('Set2', len(tag_names)))
plt.title('Gerçek Etiket Dağılımı', fontsize=14)
plt.xticks(rotation=90)
plt.ylabel('Soru Sayısı', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig(os.path.join(model_dir, 'tag_distributions.png'))
plt.show()

In [ ]:
# #32 Etiket sayısı dağılımı görselleştirme
actual_tag_counts = np.sum(y_test, axis=1)
predicted_tag_counts = np.sum(y_pred, axis=1)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(actual_tag_counts, kde=True, bins=10, color=colors[0])
plt.title('Gerçek Etiket Sayısı Dağılımı', fontsize=14)
plt.xlabel('Etiket Sayısı', fontsize=12)
plt.ylabel('Soru Sayısı', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)

plt.subplot(1, 2, 2)
sns.histplot(predicted_tag_counts, kde=True, bins=10, color=colors[2])
plt.title('Tahmin Edilen Etiket Sayısı Dağılımı', fontsize=14)
plt.xlabel('Etiket Sayısı', fontsize=12)
plt.ylabel('Soru Sayısı', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig(os.path.join(model_dir, 'tag_count_distributions.png'))
plt.show()

In [ ]:
# #33 ROC eğrileri oluşturma
from sklearn.metrics import roc_curve, auc

plt.figure(figsize=(12, 10))
for i, tag_idx in enumerate(top_tag_indices[:5]):
    tag = tag_names[tag_idx]
    if hasattr(final_model.estimators_[tag_idx], 'predict_proba'):
        try:
            y_score = final_model.estimators_[tag_idx].predict_proba(X_test_tfidf)[:,1]
            fpr, tpr, _ = roc_curve(y_test[:, tag_idx], y_score)
            roc_auc = auc(fpr, tpr)
            
            plt.plot(fpr, tpr, lw=2, label=f'{tag} (AUC = {roc_auc:.2f})')
        except Exception as e:
            print(f"ROC eğrisi {tag} için hesaplanamadı: {e}")

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Yanlış Pozitif Oranı', fontsize=14)
plt.ylabel('Doğru Pozitif Oranı', fontsize=14)
plt.title('Etiketler için ROC Eğrileri', fontsize=16)
plt.legend(loc="lower right", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig(os.path.join(model_dir, 'roc_curves.png'))
plt.show()

In [ ]:
# #34 Etiket tahmin fonksiyonu oluşturma
def predict_tags(text, model, vectorizer, tag_names, top_n=5):
    text_vector = vectorizer.transform([text])
    
    pred_probs = []
    for i, estimator in enumerate(model.estimators_):
        if hasattr(estimator, 'predict_proba'):
            try:
                prob = estimator.predict_proba(text_vector)[0][1]
                pred_probs.append((tag_names[i], prob))
            except:
                pred_probs.append((tag_names[i], 0))
        else:
            pred = estimator.predict(text_vector)[0]
            pred_probs.append((tag_names[i], float(pred)))
    
    sorted_preds = sorted(pred_probs, key=lambda x: x[1], reverse=True)
    return sorted_preds[:top_n]

In [ ]:
# #35 Örnek sorular ile tahmin testi
example_questions = [
    "How do I sort a list in Python?",
    "What's the difference between var, let and const in JavaScript?",
    "How to center a div using CSS?",
    "Getting NullPointerException in Java",
    "Best practices for SQL query optimization"
]

print("\n14. Örnek tahminler gösteriliyor...")
for i, question in enumerate(example_questions):
    print(f"\nÖrnek {i+1}: {question}")
    predictions = predict_tags(question, final_model, tfidf_vectorizer, tag_names)
    print("Tahmin edilen etiketler:")
    for tag, prob in predictions:
        print(f"  - {tag}: {prob:.4f}")

In [ ]:
# #36 Son değerlendirme
print("\n=============================================================")
print("RandomForest model eğitimi ve değerlendirmesi tamamlandı!")
print("=============================================================")
print(f"Ortalama F1 skoru: {metrics_df['F1-Score'].mean():.4f}")
print(f"Hamming Loss: {h_loss:.4f}")
print(f"Eğitim süresi: {train_time:.2f} saniye")
print(f"Model dosyası: {model_path}")
print("=============================================================")